In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
OL_phase = np.load('../data/OutLoop/OL_Phase.npy')
OL_amp = np.load('../data/OutLoop/OL_Magnitude.npy')
OL_e = np.load('../data/OutLoop/OL_Energy.npy')
IL_phase = np.load('../data/OutLoop/IL_Phase.npy')
IL_amp = np.load('../data/OutLoop/IL_Magnitude.npy')
IL_e = np.load('../data/OutLoop/IL_energy.npy')
Laser_amp = np.load('../data/OutLoop/Laser_Amp.npy')
Laser_phase = np.load('../data/OutLoop/Laser_Phs.npy')

In [ ]:
y1 = np.load('../data/OutLoop/CameraFit.npy')
#y2 = np.load('../data/OutLoop/CameraProj.npy')

In [ ]:
#x = np.concatenate((OL_phase, OL_amp,OL_e,IL_phase,IL_amp,IL_e,Laser_phase,Laser_amp), axis=0)
x = np.concatenate((OL_phase,OL_e,Laser_amp), axis=0)
X = x.reshape([y1.shape[0],-1])

In [ ]:
Y = y1.reshape([y1.shape[0],-1])

## normalize data

In [ ]:
mu_X = np.mean(X, axis=0)
sigma_X = np.std(X, axis=0)
_X = (X - mu_X)/sigma_X

In [ ]:
mu_Y = np.mean(Y, axis=0)
sigma_Y = np.std(Y, axis=0)
_Y = (Y - mu_Y) / sigma_Y

In [ ]:
_X.shape,_Y.shape

In [ ]:
print('rms focus distance',np.format_float_scientific(sigma_Y, precision=2))

### Make pytorch datasets based on the data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from pytorchtools import EarlyStopping

In [ ]:
class PhasedegDataset(Dataset):

    def __init__(self, X, Y):
        self.len = X.shape[0]
        self.X = torch.from_numpy(X)
        self.Y = torch.from_numpy(Y)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

    def __len__(self):
        return self.len

In [ ]:
phase_deg_dataset = PhasedegDataset(_X, _Y)

In [ ]:
total_num = len(phase_deg_dataset)

vali_num = int(0.1 * total_num)

train_num = total_num - vali_num
train_num, vali_num

In [ ]:
train_dataset, vali_dataset = random_split(phase_deg_dataset, [train_num, vali_num])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### make a model

In [ ]:
class FocusModel(nn.Module):
    # Phase to image model

    def __init__(self):
        super(FocusModel, self).__init__()
        self.fc1 = nn.Linear(_X.shape[1], 30)
        self.fc2 = nn.Linear(30, 20)
        self.fc3 = nn.Linear(20, _Y.shape[1])

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
net = FocusModel()
net = net.float()
net.to(device)

In [ ]:
neurals_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(neurals_num)

### loss function and optimazer

In [ ]:
import torch.optim as optim
from pytorchtools import EarlyStopping

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

### train the data

In [ ]:

def train_model(model, batch_size, patience, n_epochs):
    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = [] 
        
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in range(1, n_epochs + 1):

        model.train() 
        for i, data in enumerate(train_loader, 0):
            inputs, targets = data[0].float().to(device), data[1].float().to(device)        
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = torch.sqrt(criterion(outputs, targets))*sigma_Y[0]
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval() 
        for data in valid_loader:
            inputs, targets = data[0].float().to(device), data[1].float().to(device)
            outputs = model(inputs)
            loss = torch.sqrt(criterion(outputs, targets))*sigma_Y[0]
            valid_losses.append(loss.item())

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg)
        
        train_losses = []
        valid_losses = []

        with open('train_loss.npy', 'wb') as f:
            np.save(f, avg_train_losses)
        with open('valid_loss.npy', 'wb') as f:
            np.save(f, avg_valid_losses)
        
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))

    return  model, avg_train_losses, avg_valid_losses

In [ ]:
batch_size = 2
n_epochs = 1000
patience = 15
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=vali_dataset, batch_size=batch_size, shuffle=True)

model, train_loss, valid_loss = train_model(net, batch_size, patience, n_epochs)

In [ ]:
train_losses = np.load('train_loss.npy')
valid_losses = np.load('valid_loss.npy')

fs = 24
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(train_losses,label = 'train loss')
ax.plot(valid_losses, label = 'valid loss')

ax.set_xlabel('training epoch',fontsize = fs)
ax.set_ylabel('RMS error [meter]',fontsize = fs)
plt.grid(True)
plt.xticks(fontsize=fs-2 )
plt.yticks(fontsize=fs-2 )
plt.legend(fontsize = fs,loc='best')